In [1]:
import os
import sys
from pathlib import Path

# Force correct project root
project_root = Path("C:/Users/tilak/Desktop/TEXT-SUMMARIZER").resolve()
os.chdir(project_root)
print("Current working directory set to:", os.getcwd())

# Add the `src` folder to Python path
src_path = project_root / "src"
if str(src_path) not in sys.path:
    sys.path.insert(0, str(src_path))

print("SRC path added to sys.path:", src_path)


Current working directory set to: C:\Users\tilak\Desktop\Text-Summarizer
SRC path added to sys.path: C:\Users\tilak\Desktop\Text-Summarizer\src


In [2]:
%pwd

'C:\\Users\\tilak\\Desktop\\Text-Summarizer'

In [3]:
from dataclasses import dataclass
from pathlib import Path
@dataclass(frozen=True)
class ModelTrainerConfig:
    root_dir: Path
    data_path: Path
    model_name: Path
    epochs: int
    warmup_steps: int
    weight_decay: float
    batch_size: int
    logging_steps:int
    evaluation_strategy: str
    eval_steps: int
    save_steps: int
    gradient_accumulation_steps: int


In [4]:
from textsummarizer.constants import *
from textsummarizer.utils.common import read_yaml, create_directories

In [ ]:
class ConfigurationManager:
    def __init__(self,
                 config_filepath=CONFIG_FILE_PATH,
                 params_filepath=PARAMS_FILE_PATH):
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        create_directories([self.config.artifacts_root])

    def get_model_trainer_config(self)-> ModelTrainerConfig:
        config = self.config.model_tainer
        params =self.params.TrainingArguments
        create_directories([config.root_dir])
        model_trainer_config = ModelTrainerConfig(
            root_dir = config.root_dir,
            data_path =config.data_path,
            model_name = config.model_name,
            epochs = params.epochs,
            warmup_steps= params.warmup_steps,
            weight_decay = params.weight_decay,
            batch_size = params.batch_size,
            logging_steps = params.logging_steps,
            evaluation_strategy = params.evaluation_strategy,
            eval_steps = params.eval_steps,
            save_steps = params.save_steps,
            gradient_accumulation_steps = params.gradient_accumulation_steps
        )
        return model_trainer_config

In [6]:
from transformers import TrainingArguments,Trainer
from transformers import DataCollatorForSeq2Seq,AutoModelForSeq2SeqLM,AutoTokenizer
from datasets import load_from_disk,load_dataset
import torch


c:\Users\tilak\.conda\envs\textSM\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [ ]:
class Modeltrainer:
    def __init__(self,config:ModelTrainerConfig):
        self.config= config
    
    def train(self):
        device = 'cuda' if torch.cuda.is_available() else 'cpu'
        tokenizer = AutoTokenizer.from_pretrained(self.config.model_name)
        model_pegasus= AutoModelForSeq2SeqLM.from_pretrained(self.config.model_name)
        seq2seq_data_collator = DataCollatorForSeq2Seq(tokenizer,model=model_pegasus)
        #Loading the dataset
        dataset = load_from_disk(self.config.data_path)
        trainer_args = TrainingArguments(
            output_dir=self.config.root_dir, num_train_epochs=self.config.epochs, warmup_steps=self.config.warmup_steps,
            per_device_train_batch_size=self.config.batch_size, per_device_eval_batch_size=self.config.batch_size,
            weight_decay=self.config.weight_decay, logging_steps=self.config.logging_steps,
            eval_strategy=self.config.evaluation_strategy, eval_steps=self.config.eval_steps, save_steps=1e6,
            gradient_accumulation_steps=self.config.gradient_accumulation_steps
        ) 
        trainer = Trainer(
            model=model_pegasus,
            args=trainer_args,
            tokenizer= tokenizer,
            data_collator =seq2seq_data_collator,
            train_dataset=dataset["train"],
            eval_dataset=dataset["validation"]
        )

        trainer.train()

        model_pegasus.save_pretrained(os.path.join(self.config.root_dir,"pegasus_samsum_model"))
        tokenizer.save_pretrained(os.path.join(self.config.root_dir,"tokenizer"))

In [8]:
try:
    config = ConfigurationManager()
    model_trainer_config = config.get_model_trainer_config()
    model_trainer = Modeltrainer(config=model_trainer_config)
    model_trainer.train()
except Exception as e:
    raise e

[2025-08-24 22:16:31,475]:INFO:common: yaml file: config\config.yaml loaded successfully]
[2025-08-24 22:16:31,491]:INFO:common: yaml file: params.yaml loaded successfully]
[2025-08-24 22:16:31,495]:INFO:common: created directory at: artifacts]
[2025-08-24 22:16:31,497]:INFO:common: created directory at: artifacts/model_trainer]


Some weights of PegasusForConditionalGeneration were not initialized from the model checkpoint at google/pegasus-cnn_dailymail and are newly initialized: ['model.decoder.embed_positions.weight', 'model.encoder.embed_positions.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
C:\Users\tilak\AppData\Local\Temp\ipykernel_8260\1715238913.py:19: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(
c:\Users\tilak\.conda\envs\textSM\Lib\site-packages\torch\utils\data\dataloader.py:666: UserWarning: 'pin_memory' argument is set as true but no accelerator is found, then device pinned memory won't be used.
  warnings.warn(warn_msg)


Step,Training Loss,Validation Loss


c:\Users\tilak\.conda\envs\textSM\Lib\site-packages\transformers\modeling_utils.py:3922: UserWarning: Moving the following attributes in the config to the generation config: {'max_length': 128, 'min_length': 32, 'num_beams': 8, 'length_penalty': 0.8}. You are seeing this warning because you've set generation parameters in the model config, as opposed to in the generation config.
  warnings.warn(


MemoryError: 